# Prédiction de défaut des prêts SBA

---
# Résumé du Projet

Ce projet, construit sur les prêts historiques du programme **7(a) de la SBA** (US Small Business Administration), vise à prédire le risque de défaut (`LoanStatus`) d'une entreprise participant à ce programme. Ainsi l'objectif du modèle est de distinguer efficacement les prêts remboursés (*Paid in Full*) des pertes sèches (*Charged Off*).

Le projet est construit en plusieurs étapes :

1.  **Analyse Temporelle :** Au début du projet, les données disponibles depuis le début des années 2000 ont été analysées afin de comprendre les relations entre les différentes variables disponibles ainsi que l'évolution dans le temps des défauts. L'analyse a montré la **nécessité** de réduire la période d'étude suite aux variations des cycles macroéconomiques observées dans le temps. Ainsi, la période d'étude a été réduite à **2021-2024** afin de capter les dernières tendances économiques et éviter d'apprendre sur des données qui ne sont aujourd'hui plus d'actualité.

2.  **Enrichissement Macroéconomique :** En plus des variables disponibles dans les tables fournies par la SBA, nous avons ajouté des **variables macroéconomiques** au modèle afin de lui donner des indications sur les périodes de crise et de changement macroéconomique. Afin d'éviter tout *data leakage* possible, mais aussi dans le but de construire un projet réaliste et non seulement prédictif, ces variables macroéconomiques ont été utilisées avec un **décalage temporel** (ex : croissance du PIB de l'année précédente et non de l'année actuelle).

3.  **Validation "Business" :** Afin de capter la pertinence de ce modèle, des règles utilisées directement en entreprise ont été appliquées. Ainsi, le découpage en échantillon *train/test* respecte la temporalité et le modèle final est validé sur un échantillon **Out-of-Time (OOT)** afin de tester la stabilité des performances dans le temps.

4.  **Modélisation Hybride :** Plusieurs modèles seront comparés, en commençant par une **régression logistique** en tant que modèle benchmark, puis des modèles plus avancés (**Boosting, Bagging**) seront également testés. Différentes méthodes de sélection de variables (**Lasso, Elastic Net**) sont évaluées ainsi que différentes méthodes d'encodage des variables catégorielles (**WoE binning, One-Hot encoding**).

# Le Dataset (SBA 7(a) & 504)
Les données de départ (disponibles directement sur https://data.sba.gov/dataset/) contiennent plusieurs variables de différentes nature sur :
* **L'Emprunteur :** Localisation (`ProjectState`), Secteur d'activité (`NAICS`, `Industry`), Ancienneté.
* **La Structure du Prêt :** Montant (`GrossApproval`), Partie Garantie (`SBAGuaranteedApproval`), Durée (`TermInMonths`), Type (`RevolverStatus`).
* **Le Prêteur :** Qualité de la banque (`BankName`) et méthode d'octroi (`ProcessingMethod`).
* **Impact & Collatéral :** Emplois soutenus (`JobsSupported`) et présence de garanties (`CollateralInd`).
---

###### Importation des librairies

In [79]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import plotly.express as px


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from optbinning import OptimalBinning



from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from catboost import CatBoostClassifier, Pool, metrics


from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report

# 1. Récupération des données et premiers filtrage

In [80]:
# 2000 - 2009
df1 = pd.read_csv('Brut_Data/foia-7a-fy2000-fy2009-asof-250930.csv')

# 2010 - 2019
df2 = pd.read_csv('Brut_Data/foia-7a-fy2010-fy2019-asof-250930.csv')

# 2020 - present
df3 = pd.read_csv('Brut_Data/foia-7a-fy2020-present-asof-250930.csv')

df = pd.concat([df1, df2,  df3])


/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_3119/4182243891.py:2: DtypeWarning: Columns (14,23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Brut_Data/foia-7a-fy2000-fy2009-asof-250930.csv')
/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_3119/4182243891.py:5: DtypeWarning: Columns (14,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Brut_Data/foia-7a-fy2010-fy2019-asof-250930.csv')
/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_3119/4182243891.py:8: DtypeWarning: Columns (36,37,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('Brut_Data/foia-7a-fy2020-present-asof-250930.csv')


In [81]:
df.shape

(1583598, 43)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1583598 entries, 0 to 347513
Data columns (total 43 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   AsOfDate                     1583598 non-null  object 
 1   Program                      1583598 non-null  object 
 2   BorrName                     1583563 non-null  object 
 3   BorrStreet                   1583579 non-null  object 
 4   BorrCity                     1583598 non-null  object 
 5   BorrState                    1583598 non-null  object 
 6   BorrZip                      1583598 non-null  int64  
 7   LocationID                   1582421 non-null  float64
 8   BankName                     1583598 non-null  object 
 9   BankFDICNumber               1437677 non-null  float64
 10  BankNCUANumber               39120 non-null    float64
 11  BankStreet                   1582420 non-null  object 
 12  BankCity                     1582421 non-null  o

In [ ]:
# GrossApproval correspond au montant total du prêt
# SBAGuaranteedApproval correspond au montant garantit SBA
pd.options.display.max_columns = None
df.head(3)

In [ ]:
df.isna().sum()/df.shape[0]

In [ ]:
for col in df.columns:
    if (df[col].isna().sum()/df.shape[0]) > 0.3:
        df = df.drop(col, axis= 1)
    else:
        pass

In [ ]:
df.isna().sum()/df.shape[0]

In [ ]:
df.loc[df["GrossChargeoffAmount"]>0]["LoanStatus"].value_counts()

In [ ]:
# target 
df['LoanStatus'].value_counts()

les modalités de la variable cible:
- **PIF** = PAID in full (donc remboursement total de l'entreprise et pas de défaut)
- **CHGOFF** = CHARGED OFF (donc l'emprunteur a arrêter de payer ==> Défaut)

ON a également:
- **COMMIT** = COMMITMENT / UNDISBURSED (à supprimer car le prêt n'a pas encore été donné)
- **CANCLD** = Cancelled (le contrat est annulé et le prêt n'a pas été approuvé donc à supprimer aussi de la base)

In [ ]:
df = df.loc[(df["LoanStatus"] == 'PIF') | (df['LoanStatus'] == 'CHGOFF')]

In [ ]:
df["LoanStatus"].value_counts(normalize=True)

In [ ]:
# Pour ne pas faire de data leakage et colonnes inutiles pour le moment
df = df.drop(columns={"GrossChargeoffAmount",
                               "AsOfDate"})

In [ ]:
# On binarise la cible
df["LoanStatus"] = np.where(df["LoanStatus"] == 'PIF', 0, 1)

# 2. DATA ANALYSE

In [ ]:
pd.options.display.max_columns = None
df.head()

In [ ]:
# On garde seulement la dernière date d'obs d'un meme pret
df = df.drop_duplicates(subset=["BorrName", "BorrStreet", "BorrCity", "FirstDisbursementDate"], keep="last")
df = df.drop_duplicates()

In [ ]:
df["ApprovalDate"]= pd.to_datetime(df["ApprovalDate"], errors='coerce')
# Mise au format date
df["FirstDisbursementDate"] = pd.to_datetime(df["FirstDisbursementDate"])
df = df.set_index("ApprovalDate")

In [ ]:
df = df.drop(columns={'Program',
                      'BorrName',
                      'BorrStreet',
                      'BorrCity',
                      'LocationID',
                      'BankFDICNumber',
                      'BankZip',
                      'BankStreet',
                      'BankCity',
                      'NAICSDescription',
                      'CongressionalDistrict',
                      "BorrZip"
})

In [ ]:
print(df.shape)
df.columns

## 2.1. Valeurs manquantes

In [ ]:
# Pas de valeurs manquantes pour les variables numériques
col_numericals = ["GrossApproval", "SBAGuaranteedApproval", "TerminMonths", "JobsSupported"]
df[col_numericals].isna().sum()

In [ ]:
col_restants = [col for col in df.columns if col not in col_numericals]
df[col_restants].isna().sum()

In [ ]:
# On voit que lorsque Bankstate est manquant on a MISSINGMAINBANKID comme nom de banque (inconnu)
df.loc[df["BankState"].isna()]

# On va donc imputer ces valeurs dans une catégorie "unknown"
df["BankName"] = np.where(df["BankName"]=="MISSINGMAINBANKID", "unknown", df["BankName"])
df["BankState"] = np.where(df["BankState"].isna(), "unknown", df["BankState"])
print(f'Nombre de lignes imputées {df[df["BankState"] == "unknown"].shape[0]}')

In [ ]:
# Les lignes ou FirstDisbursementDate est manquant les autres variables sont renseignées
df.loc[df['FirstDisbursementDate'].isna()]
# Sachant que le premier versement arrive après un certain temps de la date d'approbation on peut prendre la médiane (dans tout les cas ca n'aura pas un grand impact car peu de valeurs manquantes)
approval_dates = df.index.to_series()
delai_median = (df['FirstDisbursementDate'] - approval_dates).median()
valeurs_remplacement = approval_dates + delai_median
df['FirstDisbursementDate'] = df['FirstDisbursementDate'].fillna(valeurs_remplacement)